In [ ]:
import xarray as xr
from windrose import WindroseAxes
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray
import numpy as np
import scipy
import cv2 as cv
from scipy.ndimage import rotate
from ocr.wind import apply_wind_directional_convolution, classify_wind_directions, apply_mode_calc, create_finescale_wind_direction, create_composite_bp_map
from ocr.utils import lon_to_180, interpolate_to_30, subset_region_xy, convert_coords

In [ ]:
# Load in 270m risk

In [ ]:
historical_path = "s3://carbonplan-risks/riley2025/2011climaterun"
future_path = "s3://carbonplan-risks/riley2025/2047climaterun"
riley = {}
riley['2011'] = xr.open_zarr(historical_path+'.zarr')
riley['2047'] = xr.open_zarr(future_path+'.zarr')

In [ ]:
fire_name = 'Eaton'
structures = gpd.read_file(f's3://carbonplan-risks/structures/{fire_name}/DINS_2025_{fire_name}_Public_View.shp'.format(fire_name))
structures = structures.to_crs('EPSG:5070')

In [ ]:
buffer = 20000
x_min, x_max = structures.geometry.x.min()-buffer, structures.geometry.x.max()+buffer
y_min, y_max = structures.geometry.y.min()-buffer, structures.geometry.y.max()+buffer

In [ ]:
subset = subset_region_xy(riley['2011'], [x_min, x_max], [y_min, y_max]).sel(band=1)

In [ ]:
[(lon_min, lat_max), (lon_max, lat_min)] = convert_coords([(x_min, y_max), (x_max, y_min)], from_crs='EPSG:5070', to_crs='EPSG:4326')

In [ ]:
# expand the lats a smidge to make sure lat and lon dim lengths are both >1. it fails if dim length is 1. fixing this corner-case is low-priority.
lat_max += 0.2
lat_min -= 0.2
lon_max += 0.15
lon_min -= 0.15

In [ ]:
# load in the 30m product as a template
rps_30 = rioxarray.open_rasterio('s3://carbonplan-risks/wildfirecommunities/RPS_CA.tif')

In [ ]:
rps_30 = subset_region_xy(rps_30, [x_min, x_max], [y_min, y_max]).sel(band=1)

In [ ]:
subset_30 = interpolate_to_30(subset, rps_30)

In [ ]:
# Create a BP map smeared according to each of the 8 cardinal and intercardinal directions

In [ ]:
blurred_bp = apply_wind_directional_convolution(subset_30['BP'], iterations=3)

In [ ]:
# Load in the dataset of wind for fire weather days

In [ ]:
important_days = xr.open_zarr('s3://carbonplan-risks/era5/fire_weather_days_v2.zarr')

In [ ]:
important_days = lon_to_180(important_days)

In [ ]:
subset_wind = important_days.sel(latitude=slice(lat_max, lat_min), longitude=slice(lon_min, lon_max))
wind_directions = subset_wind['sfcWindfromdir']

In [ ]:
# we only have to do the wind direction classification once - want to get the distribution of each direction. prep this and store it like `fire_weather_days_v2.zarr`

In [ ]:
direction_indices = classify_wind_directions(wind_directions).chunk(dict(time=-1))

In [ ]:
direction_modes = apply_mode_calc(direction_indices).compute()

In [ ]:
direction_modes_reprojected = create_finescale_wind_direction(blurred_bp, direction_modes)

In [ ]:
wind_informed_bp = create_composite_bp_map(blurred_bp, direction_modes_reprojected)

In [ ]:
wind_informed_bp.plot()